import libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pylab as plt
from keras.preprocessing import image
from tensorflow.keras import utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score
import time

2023-05-08 20:15:51.022454: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-08 20:15:51.064861: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-08 20:15:51.065659: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 20:15:53.021595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


functions

In [2]:
SAVED_MODEL_BEST_ACCURACY_RESULTS = []
SAVED_MODEL_BEST_LOSS_RESULTS = []
IMG_SIZE = 64
EPOCHS = 30

In [3]:
def get_image_and_transform(path):
    data = utils.load_img(path, color_mode='grayscale', target_size=(64, 64))
    data = utils.img_to_array(data)
    data = np.expand_dims(data, axis=0)
    data = data / 255.0
    return data

In [4]:
def build_dataset(dataset_dir): # build 2 datasets for training and testing
    x_train, y_train = [], []
    classes = sorted(os.listdir(dataset_dir))
    for i in range(len(classes)):
        class_dir = os.path.join(dataset_dir, classes[i])
        images = os.listdir(class_dir)
        for image in images:
            full_image_path = os.path.join(class_dir, image)
            if 'ipynb_checkpoints' in full_image_path:
                continue
            x_train.append(get_image_and_transform(full_image_path))
            y_train.append(i)
    return x_train, y_train

In [5]:
def get_examples_and_classes(dataset_dir, num): # return persons' names and persons' ears
    classes = sorted(os.listdir(dataset_dir))
    examples = []
    for i in range(len(classes)):
        class_dir = os.path.join(dataset_dir, classes[i])
        images = os.listdir(class_dir)
        examples.append(os.path.join(class_dir, images[num]))

    return classes, examples

In [6]:
def plot(accuracy, loss, val_accuracy, val_loss): # return schedule of accuracy and loss
    fig, ax1 = plt.subplots()
    ax1.set_xlabel('epoch')

    ax1.set_ylabel('total loss')
    ax1.plot(loss,color='tab:red')
    ax1.plot( val_loss, color='tab:brown')
    ax1.tick_params(axis='y')

    ax2 = ax1.twinx()  
    ax2.set_ylabel('accuracy')  
    ax2.plot( accuracy, color='tab:blue')
    ax2.plot( val_accuracy, color='tab:green')
    ax2.tick_params(axis='y')
    fig.tight_layout()

In [7]:
def plot_many(title='title', data=[]): # return schedules of accuracy
    for i,o in enumerate(data):
        plt.figure(i)
        plt.plot([i for i in range(EPOCHS)],o, label=title)
        plt.legend(loc='upper left')

In [8]:
def max_or_min_avarage_val(list_of_data, accuracy=True): #return dictionary of average values of lists
    dict_all_l = {}
    for i, o in enumerate(list_of_data):
        dict_all_l[i] = sum(o)/len(o)
    if accuracy:
        return dict(sorted(dict_all_l.items(), key=lambda item: item[1], reverse=True))
    else:
        return dict(sorted(dict_all_l.items(), key=lambda item: item[1]))

In [9]:
def save_lists(): # save lists of accuracy and loss
    t = time.ctime(time.time())[:-5]
    with open(f'accuracy {t}.txt', 'w') as f:
        for item in SAVED_MODEL_BEST_ACCURACY_RESULTS:
            f.write("%s\n" % item)
    # with open(f'loss {t}.txt', 'w') as f:
    #     for item in SAVED_MODEL_BEST_LOSS_RESULTS:
    #         f.write("%s\n" % item)

In [10]:
def read_list_from_file(filename): # read accuracy and loss data  from files
    lst = []
    with open(filename, 'r') as file:
        for line in file.readlines():
            if line in ('', ' ','\n'):
                break
            line = line[1:-2].split(', ')
            lst.append([float(i.strip()) for i in line])
    return lst

In [11]:
def show_best_res(list_of_data): # return max value in every list of accuracy
    max_val = {}
    for i, o in enumerate(list_of_data):
        max_val[i] = max(o)
    return dict(sorted(max_val.items(), key=lambda item: item[1], reverse=True))

dataset

In [12]:
train_folder = "/home/jovyan/dataset/dataset2/train"
test_folder = "/home/jovyan/dataset/dataset2/test"

In [17]:
train_datagen = ImageDataGenerator()#rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator()#rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_folder,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 color_mode='grayscale')

test_set = test_datagen.flow_from_directory(test_folder,
                                            target_size = (64, 64),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            color_mode='grayscale')

# print(training_set.filepaths)
# print(training_set.classes)
# print(training_set.class_indices)


Found 22736 images belonging to 164 classes.
Found 5683 images belonging to 164 classes.


model

In [18]:
def model(in_1=8, in_2=8, in_3=8, in_4=8):
    model = Sequential()
    model.add(Conv2D(filters=in_1,kernel_size=3, strides=(1,1), input_shape=(64,64,1), padding='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(BatchNormalization())
    

    model.add(Conv2D(filters=in_2,kernel_size=3, strides=1, padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=in_3,kernel_size=2, strides=1, padding ='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(BatchNormalization())
    
    
    model.add(Conv2D(filters=in_4,kernel_size=2, strides=1, padding ='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(BatchNormalization())

    model.add(Flatten())
    # model.add(tf.keras.layers.Dropout(0.1))
    model.add(Dense(units = 512 , activation = 'tanh'))
    # model.add(Dense(units = 1))
    model.add(Dense(units = 164 , activation = 'softmax'))

    model.compile( optimizer = 'Adam', loss = 'categorical_crossentropy',metrics = ['accuracy']) # optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3)

    return model
summary_example = model()
summary_example.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 64, 64, 8)         80        
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 32, 32, 8)        0         
 2D)                                                             
                                                                 
 batch_normalization_8 (Batc  (None, 32, 32, 8)        32        
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 32, 32, 8)         584       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 16, 16, 8)        0         
 2D)                                                             
                                                      

choose best hyperparameters

In [19]:
def model_and_best_hyperparameters(nums):
    start_time1 = time.time() 
    for i in nums:
        start_time2 = time.time()
        
        ready_model = model(i,i*2,i*3, i*4)        # change order 
        trained_model = ready_model.fit(training_set, steps_per_epoch = 32, epochs = EPOCHS, validation_data = test_set)
        
        print("for one training time taken: {:.2f} seconds".format(time.time() - start_time2)) # how much time one training take
        
        accuracy = trained_model.history['accuracy']
        loss = trained_model.history['loss']

        SAVED_MODEL_BEST_ACCURACY_RESULTS.append(accuracy)
        SAVED_MODEL_BEST_LOSS_RESULTS.append(loss)
        
    print("for all training time taken: {:.2f} seconds".format(time.time() - start_time1)) # how much time all training take

In [20]:
model_and_best_hyperparameters([16,32])

Epoch 1/30


2023-05-08 20:18:42.182383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - ETA: 0s - loss: 5.3529 - accuracy: 0.0186

2023-05-08 20:18:45.826146: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - 9s 234ms/step - loss: 5.3529 - accuracy: 0.0186 - val_loss: 5.5004 - val_accuracy: 0.0065
Epoch 2/30
32/32 [==============================] - 7s 215ms/step - loss: 5.1267 - accuracy: 0.0303 - val_loss: 5.3333 - val_accuracy: 0.0125
Epoch 3/30
32/32 [==============================] - 6s 200ms/step - loss: 4.9869 - accuracy: 0.0391 - val_loss: 5.2824 - val_accuracy: 0.0151
Epoch 4/30
32/32 [==============================] - 7s 235ms/step - loss: 4.8379 - accuracy: 0.0654 - val_loss: 4.9579 - val_accuracy: 0.0310
Epoch 5/30
32/32 [==============================] - 7s 224ms/step - loss: 4.6993 - accuracy: 0.0674 - val_loss: 4.8629 - val_accuracy: 0.0377
Epoch 6/30
32/32 [==============================] - 7s 222ms/step - loss: 4.5379 - accuracy: 0.0801 - val_loss: 4.9350 - val_accuracy: 0.0399
Epoch 7/30
32/32 [==============================] - 7s 223ms/step - loss: 4.5584 - accuracy: 0.0781 - val_loss: 4.8561 - val_accuracy: 0.0475
Epoch 8/30
32/32 

2023-05-08 20:22:13.372402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - ETA: 0s - loss: 5.5021 - accuracy: 0.0195

2023-05-08 20:22:19.439347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - 14s 414ms/step - loss: 5.5021 - accuracy: 0.0195 - val_loss: 5.7314 - val_accuracy: 0.0083
Epoch 2/30
32/32 [==============================] - 13s 402ms/step - loss: 5.1950 - accuracy: 0.0273 - val_loss: 5.4354 - val_accuracy: 0.0148
Epoch 3/30
32/32 [==============================] - 12s 372ms/step - loss: 5.0346 - accuracy: 0.0371 - val_loss: 5.3360 - val_accuracy: 0.0157
Epoch 4/30
32/32 [==============================] - 10s 332ms/step - loss: 4.8986 - accuracy: 0.0518 - val_loss: 4.9061 - val_accuracy: 0.0391
Epoch 5/30
32/32 [==============================] - 10s 327ms/step - loss: 4.7607 - accuracy: 0.0645 - val_loss: 5.3730 - val_accuracy: 0.0183
Epoch 6/30
32/32 [==============================] - 11s 358ms/step - loss: 4.6403 - accuracy: 0.0742 - val_loss: 4.9192 - val_accuracy: 0.0408
Epoch 7/30
32/32 [==============================] - 13s 413ms/step - loss: 4.4242 - accuracy: 0.0947 - val_loss: 5.1190 - val_accuracy: 0.0373
Epoch 8/30

In [22]:
print("sorted list of average  epochs' values for accuracy  ", max_or_min_avarage_val(SAVED_MODEL_BEST_ACCURACY_RESULTS))
print("sorted list of max  epochs' values for accuracy  ", show_best_res(SAVED_MODEL_BEST_ACCURACY_RESULTS))
# save_lists()

sorted list of average  epochs' values for accuracy   {0: 0.20856119791666666, 1: 0.2078125}
sorted list of max  epochs' values for accuracy   {0: 0.4033203125, 1: 0.40234375}


In [ ]:
# file = 'accuracy Sat May  6 11:19:23.txt'
# l = read_list_from_file(file)
# print("sorted list of max  epochs' values for accuracy  ", show_best_res(l))

train and save model

In [ ]:
def final_model():
    ready_model = model(8,16,32,64)
    call_back = ModelCheckpoint(f"/home/jovyan/weights.hdf5", monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
    trained_model = ready_model.fit(training_set, steps_per_epoch = 32, epochs = 30, validation_data = test_set, callbacks = [call_back])
    return trained_model


In [ ]:
trained_model = final_model()

plot model

In [ ]:
# plot_many('accuracy', SAVED_MODEL_BEST_ACCURACY_RESULTS)


In [ ]:
if trained_model:
    plot(trained_model.history['accuracy'], trained_model.history['loss'], trained_model.history['val_accuracy'], trained_model.history['val_loss'])

predict

In [ ]:
# predict_model = Sequential()
# for layer in model.layers[:-1]:
#     predict_model.add(layer)
# predict_model.add(Dense(units=1, activation=None))
# predict_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
# predict_model.summary()

predict_model = model(8,16,32,64)
predict_model.summary()
predict_model.load_weights('weights.hdf5')#, skip_mismatch=True, by_name=True)

# # Load weights and biases from model manually
# for i in range(len(predict_model.layers) - 1):
#     predict_model.layers[i].set_weights(model.layers[i].get_weights())


In [ ]:
def predict(model, known_data_path, unknown_data_path):
    known_data = get_image_and_transform(known_data_path)
    unknown_data = get_image_and_transform(unknown_data_path)

    known_data = model.predict(known_data)
    unknown_data = model.predict(unknown_data)
    distance = unknown_data - known_data

    threshold = 0.018614814
    return  distance[0][0] < threshold, distance[0][0]

predict(predict_model, '/home/jovyan/12.jpg', '/home/jovyan/11.jpg')

plot prediction

In [ ]:
classes, examples1 = get_examples_and_classes(test_folder,1)
classes, examples2 = get_examples_and_classes(test_folder,2)
y_true = [i for i in range(len(classes))]
y_predict_tensors = []
y_predict_labels = []

for i in range(len(classes)):
    predict_label, predict_tensor = predict(predict_model, examples1[i], examples2[i])
    y_predict_tensors.append(predict_tensor)
    y_predict_labels.append(predict_label)    

In [ ]:
print(classification_report(y_true, y_predict_labels, target_names=classes))

choose best threshold

In [ ]:
thresholds = np.linspace(0.0001, 0.5, 1000)

# Calculate F1 score for each threshold
f1_scores = []
for threshold in thresholds:
    y_pred = (y_predict_tensors < threshold).astype(int)
    f1_scores.append(f1_score(y_true, y_pred, average='micro'))

# Choose the threshold that maximizes F1 score
best_threshold = thresholds[np.argmax(f1_scores)]
best_threshold